# Nightmare: Alias Reuse

This is a "nightmare" notebook to show the horrible dangers of alias reuse in a Jupyter (iPython) Notebook.

Do __not__ do what you see here. Learn it, recognise it, and then go do the opposite!

Stay safe out there, y'all!

In [14]:
# Let's make some objects

import time;

def get_random_number():
    """Provide a 'random' number in the range [0, 100] doubly inclusive.
    
    Returns
        int: A 'random' integer based on computer time.
    """
    random_number = time.localtime().tm_sec;
    random_number = ((random_number * 37) % 101);
    return (random_number);
# fed

random = get_random_number();

center = 5;

noise = center + random;

print("True Center:", center);
print("Noisy Center:", noise);

True Center: 5
Noisy Center: 75


Not too terrible so far, we made a "random" number that's generated from a function using the computer time, and then we shifted our `center` value by that "random" amount. You can actually keep re-running that cell, above, and see the numbers changing.

But... we chose a terrible name, `random`, which is way too generic and we're gonna run into alias stomping.

Especially because the `random` package is a base Python package that is used for random number generation and probabilistic models.

Let's say we took the above and split it apart into two cells, to keep our function separate from our test code, and let's try a little bit of closure "magic" to spice things up:

In [27]:
# Same as above, almost, but now we've defined a self-referential closure...

import time;

random = 37;

def get_random_number():
    """Provide a 'random' number in the range [0, 100] doubly inclusive.
    
    Returns
        int: A 'random' integer based on computer time.
    """
    random_number = time.localtime().tm_sec;
    random_number = ((random_number * random) % 101);
    print("DEBUG | random:", random);
    return (random_number);
# fed

In [35]:
random = get_random_number();

center = 5;

noise = center + random;

print("True Center:", center);
print("Noisy Center:", noise);

DEBUG | random: 72
True Center: 5
Noisy Center: 70


Jinkies! What fresh new hell is this!?!?

We defined `random`, then we defined a function below it that references it out of scope (using a closure), and then in our next cell we reassign it to the output of that function. Yikes!

Makes things a bit more "random"-esque ... kinda. Not really, since it's just a bit of shifting, but kinda yes, because it's based on runtime and the previous runtime and all the previous runtimes before that. So yeah, maybe, kinda neat, right?

_Wrong!!!_

Hellscape!!!

This is super gross. This is a nightmare. We've now got a circular dependency in our code and we've alias stomped ourselves. Sure, it's on purpose, but now let's just make it really friggin' terrible:

In [36]:
import random;

their_random = int(random.uniform(0, 100));
my_random = get_random_number();

print("Theirs:", their_random);
print("  Ours:", my_random);

TypeError: unsupported operand type(s) for *: 'int' and 'module'

__Blorp!__

What failed?! Oh, right! We chose a really terrible alias name of `random`, and then jammed that through a closure into our function, and then later we imported the Python package which gladly stomped on that alias, but then when went to use it inside the function, it failed horrendously because now `random` points to the import Module `random` and not our last assignment to the integer value `random`.

Notice too that the code fails in the `get_random_number()` call which came after the `random.uniform()` call. So the value of `their_random` actually succeeded, but the value of `my_random` failed. Can we check this? Of course!

Let the nightmare continue!!!

In [37]:
print("Theirs:", their_random);

Theirs: 33


In [38]:
print("  Ours:", my_random);

NameError: name 'my_random' is not defined

> _O ill-dispersing wind of misery!_<br>
> _O my accursèd womb, the bed of death!_<br>
> _A cockatrice hast thou hatched to the world,_<br>
> _Whose unavoided eye is murderous._

Let us end this horrible charade with that.

___Don't do this!___

Mixing alias stomping and closures is gonna be a nightmare. Don't do it. Save yourself, save your sanity. Don't do it.